# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО: Никишин Евгений Сергеевич

Группа: 317

In [21]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import KFold
import scipy as sp
import time

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [22]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [8]:
data.shape

(32769, 10)

In [9]:
# доля положительных примеров
data.ACTION.mean()

0.94210992096188473

In [10]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [23]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния (учитите, что он должен возвращать вероятность — отношение объектов первого класса среди соседей к числу соседей). Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba).

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

Какая функция расстояния оказалась лучшей?

#### Опишем класс kNN, как в sklearn'e

In [24]:
class kNN:
    def __init__(self, distance_function, n_neighbors):
        self.distance_function = distance_function
        self.n_neighbors = n_neighbors
    def predict(self, X_train, X_test, y_train):
        distance_matrix = self.distance_function(X_train, X_test)
        nearest = np.argsort(distance_matrix, axis=0).T[:, :self.n_neighbors]
        predict_proba = np.sum(y_train[nearest], axis=1).astype(float) / self.n_neighbors
        return predict_proba

In [25]:
def match_distance(X, Y):
    return np.sum(X[:, :, np.newaxis] != Y.T[np.newaxis, :, :], axis=1)

#### Для второй метрики заранее посчитаем все необходимое

In [26]:
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
y_train = y_train.as_matrix()
y_test = y_test.as_matrix()

In [31]:
def count_weights():
    weights = np.zeros(X_train.shape)
    for column in range(X_train.shape[1]):
        uniques, counts = np.unique(X_train[:, column], return_counts=True)
        l = X_train.shape[0]
        p_2_sorted = (pd.Series((counts * (counts-1)).astype(float) / (l * (l-1)), index=uniques))
        p_2_sorted.sort()
        for i in range(X_train.shape[0]):
            weights[i, column] = np.sum(p_2_sorted[0:np.where(p_2_sorted.index == X_train[i, column])[0][-1]+1])
    return weights

In [32]:
smoothed_weights = count_weights()

In [33]:
def smoothed_match_distance_row(X, y):
    not_equal_component = np.sum(X != y, axis=1)
    equal_component = np.sum((X == y) * (smoothed_weights), axis=1)
    return not_equal_component + equal_component

In [34]:
def smoothed_match_distance(X, Y):
    dist = np.zeros((X.shape[0], Y.shape[0]))
    for i in range(Y.shape[0]):
        dist[:, i] = smoothed_match_distance_row(X, Y[i, :])
    return dist

#### Для третьей тоже все необходимое считаем заранее

In [36]:
def count_log_weights():
    list_logarithms = []
    weights = np.zeros(X_train.shape)
    for column in range(X_train.shape[1]):
        uniques, counts = np.unique(X_train[:, column], return_counts=True)
        logarithms = np.log(counts)
        logarithms[logarithms == -np.inf] = 0
        logarithms = pd.Series(logarithms, index=uniques)
        weights[:, column] = logarithms[X_train[:, column]]
        list_logarithms.append(logarithms)
    return weights, list_logarithms

In [37]:
log_weights, list_logarithms = count_log_weights()

In [38]:
def log_match_distance_row(X, y):
    not_equal = X != y
    dist = np.zeros((X.shape[0]))
    for column in range(X.shape[1]):
        current_logarithms = list_logarithms[column]
        try: # если в тесте окажется несуществующее значение признака
            current_log = current_logarithms[y[column]]
            dist += not_equal[:, column] * log_weights[:, column] * current_log
        except KeyError:
            pass
    return dist

In [39]:
def log_match_distance(X, Y):
    dist = np.zeros((X.shape[0], Y.shape[0]))
    for i in range(Y.shape[0]):
        dist[:, i] = log_match_distance_row(X, Y[i, :])
    return dist

#### Сравним результаты

In [40]:
clf = kNN(match_distance, 10)
predict = clf.predict(X_train, X_test, y_train)
print 'AUC-ROC на тесте для первой метрики:', roc_auc_score(y_test, predict)

AUC-ROC на тесте для первой метрики: 0.83088009598


In [41]:
clf = kNN(smoothed_match_distance, 10)
predict = clf.predict(X_train, X_test, y_train)
print 'AUC-ROC на тесте для второй метрики:', roc_auc_score(y_test, predict)

AUC-ROC на тесте для второй метрики: 0.832857727689


In [42]:
clf = kNN(log_match_distance, 10)
predict = clf.predict(X_train, X_test, y_train)
print 'AUC-ROC на тесте для третьей метрики:', roc_auc_score(y_test, predict)

AUC-ROC на тесте для третьей метрики: 0.817053027738


### Вывод: лучшей метрикой оказался сглаженный индикатор совпадения

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

#### Без фолдинга

In [141]:
data_processed = pd.DataFrame()
data_processed['ACTION'] = data['ACTION']

for column in X_train.columns:
    indices_with_values = X_train[column].value_counts()
    counts = indices_with_values[data[column]]
    counts = counts.fillna(0)
    counts = counts.as_matrix()
    data_processed[column+'COUNTS'] = counts
    
    indices_with_values = X_train[y_train == 1][column].value_counts()
    clicks = indices_with_values[data[column]]
    clicks = clicks.fillna(0)
    clicks = clicks.as_matrix()
    data_processed[column+'CLICKS'] = clicks
    
    data_processed[column+'RATIO'] = (clicks + 1) / (counts + 2)
    data_processed = data_processed.fillna(0)
    
X_train_pro, X_test_pro, y_train_pro, y_test_pro = train_test_split(
    data_processed.iloc[:, 1:], data_processed.iloc[:, 0], test_size=0.3, random_state=241)

In [142]:
k_values = [1, 2, 4, 8, 16, 32]
best_auc = 0
best_k = 0
for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train_pro, y_train_pro)
    predict = clf.predict_proba(X_test_pro)
    auc = roc_auc_score(y_test_pro.astype(float), predict[:, 1])
    if auc > best_auc:
        best_auc = auc
        best_k = k
print "Наилучшее качество:", best_auc, "Достигается при числе соседей k =", best_k

Наилучшее качество: 0.794899311419 Достигается при числе соседей k = 16


#### С фолдингом

In [164]:
n_folds = 50
data_processed_fold = pd.DataFrame()
data_processed_fold['ACTION'] = data['ACTION']

for column in X_train.columns:
    data_processed_fold[column+'COUNTS'] = np.zeros(data[column].shape)
    data_processed_fold[column+'CLICKS'] = np.zeros(data[column].shape)
    data_processed_fold[column+'RATIO'] = np.zeros(data[column].shape)
    
kf = KFold(X_train.shape[0], n_folds=n_folds)
for train_index, test_index in kf:
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = \
        X_train.iloc[train_index], X_train.iloc[test_index], y_train.iloc[train_index], y_train.iloc[test_index]
        
    for column in X_train.columns:
        indices_with_values = X_train_temp[column].value_counts()
        counts = indices_with_values[X_test_temp[column]]
        counts = counts.fillna(0)
        counts = counts.as_matrix()
        data_processed_fold.loc[X_test_temp.index, [column+'COUNTS']] = counts
        
        indices_with_values = X_train_temp[y_train_temp == 1][column].value_counts()
        clicks = indices_with_values[X_test_temp[column]]
        clicks = clicks.fillna(0)
        clicks = clicks.as_matrix()
        data_processed_fold.loc[X_test_temp.index, [column+'CLICKS']] = clicks
        
for column in X_train.columns:
    indices_with_values = X_train[column].value_counts()
    counts = indices_with_values[X_test[column]]
    counts = counts.fillna(0)
    counts = counts.as_matrix()
    data_processed_fold.loc[X_test.index, [column+'COUNTS']] = counts
    
    indices_with_values = X_train[y_train == 1][column].value_counts()
    clicks = indices_with_values[X_test[column]]
    clicks = clicks.fillna(0)
    clicks = clicks.as_matrix()
    data_processed_fold.loc[X_test.index, [column+'CLICKS']] = clicks
    
for column in X_train.columns:
    data_processed_fold[column+'RATIO'] = (
        data_processed_fold[column+'CLICKS'] + 1) / (data_processed_fold[column+'COUNTS'] + 2)
    data_processed_fold = data_processed_fold.fillna(0)
    
X_train_pro_fold, X_test_pro_fold, y_train_pro_fold, y_test_pro_fold = train_test_split(
    data_processed_fold.iloc[:, 1:], data_processed_fold.iloc[:, 0], test_size=0.3, random_state=241)

In [166]:
k_values = [1, 2, 4, 8, 16, 32, 64]
best_auc = 0
best_k = 0
for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train_pro_fold, y_train_pro_fold)
    predict = clf.predict_proba(X_test_pro_fold)
    auc = roc_auc_score(y_test_pro_fold.astype(float), predict[:, 1])
    if auc > best_auc:
        best_auc = auc
        best_k = k
print "Наилучшее качество:", best_auc, "Достигается при числе соседей k =", best_k

Наилучшее качество: 0.768569466639 Достигается при числе соседей k = 16


#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

In [7]:
data_with_pairs = data.copy()
for i in range(1, data.shape[1]):
    for j in range(i+1, data.shape[1]):
        column1 = data.columns[i]
        column2 = data.columns[j]
        data_with_pairs[column1 + '+' + column2] = data[column1] * 10**6 + data[column2]

In [8]:
data_with_pairs.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,...,ROLE_DEPTNAME+ROLE_TITLE,ROLE_DEPTNAME+ROLE_FAMILY_DESC,ROLE_DEPTNAME+ROLE_FAMILY,ROLE_DEPTNAME+ROLE_CODE,ROLE_TITLE+ROLE_FAMILY_DESC,ROLE_TITLE+ROLE_FAMILY,ROLE_TITLE+ROLE_CODE,ROLE_FAMILY_DESC+ROLE_FAMILY,ROLE_FAMILY_DESC+ROLE_CODE,ROLE_FAMILY+ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908,...,123472117905,123472117906,123472290919,123472117908,117905117906,117905290919,117905117908,117906290919,117906117908,290919117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539,...,123125118536,123125118536,123125308574,123125118539,118536118536,118536308574,118536118539,118536308574,118536118539,308574118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880,...,117884117879,117884267952,117884019721,117884117880,117879267952,117879019721,117879117880,267952019721,267952117880,19721117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322,...,119993118321,119993240983,119993290919,119993118322,118321240983,118321290919,118321118322,240983290919,240983118322,290919118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325,...,119569119323,119569123932,119569019793,119569119325,119323123932,119323019793,119323119325,123932019793,123932119325,19793119325


#### Без фолдинга

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_with_pairs.iloc[:, 1:], data_with_pairs.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

In [10]:
data_processed = pd.DataFrame()
data_processed['ACTION'] = data_with_pairs['ACTION']

for column in X_train.columns:
    indices_with_values = X_train[column].value_counts()
    counts = indices_with_values[data_with_pairs[column]]
    counts = counts.fillna(0)
    counts = counts.as_matrix()
    data_processed[column+'COUNTS'] = counts
    
    indices_with_values = X_train[y_train == 1][column].value_counts()
    clicks = indices_with_values[data_with_pairs[column]]
    clicks = clicks.fillna(0)
    clicks = clicks.as_matrix()
    data_processed[column+'CLICKS'] = clicks
    
    data_processed[column+'RATIO'] = (clicks + 1) / (counts + 2)
    data_processed = data_processed.fillna(0)
    
X_train_pro, X_test_pro, y_train_pro, y_test_pro = train_test_split(
    data_processed.iloc[:, 1:], data_processed.iloc[:, 0], test_size=0.3, random_state=241)

In [11]:
k_values = [1, 2, 4, 8, 16, 32]
best_auc = 0
best_k = 0
for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train_pro, y_train_pro)
    predict = clf.predict_proba(X_test_pro)
    auc = roc_auc_score(y_test_pro.astype(float), predict[:, 1])
    if auc > best_auc:
        best_auc = auc
        best_k = k
print "Наилучшее качество:", best_auc, "Достигается при числе соседей k =", best_k

Наилучшее качество: 0.801859076241 Достигается при числе соседей k = 8


#### С фолдингом

In [12]:
n_folds = 50
data_processed_fold = pd.DataFrame()
data_processed_fold['ACTION'] = data_with_pairs['ACTION']

for column in X_train.columns:
    data_processed_fold[column+'COUNTS'] = np.zeros(data_with_pairs[column].shape)
    data_processed_fold[column+'CLICKS'] = np.zeros(data_with_pairs[column].shape)
    data_processed_fold[column+'RATIO'] = np.zeros(data_with_pairs[column].shape)
    
kf = KFold(X_train.shape[0], n_folds=n_folds)
for train_index, test_index in kf:
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = \
        X_train.iloc[train_index], X_train.iloc[test_index], y_train.iloc[train_index], y_train.iloc[test_index]
        
    for column in X_train.columns:
        indices_with_values = X_train_temp[column].value_counts()
        counts = indices_with_values[X_test_temp[column]]
        counts = counts.fillna(0)
        counts = counts.as_matrix()
        data_processed_fold.loc[X_test_temp.index, [column+'COUNTS']] = counts
        
        indices_with_values = X_train_temp[y_train_temp == 1][column].value_counts()
        clicks = indices_with_values[X_test_temp[column]]
        clicks = clicks.fillna(0)
        clicks = clicks.as_matrix()
        data_processed_fold.loc[X_test_temp.index, [column+'CLICKS']] = clicks
        
for column in X_train.columns:
    indices_with_values = X_train[column].value_counts()
    counts = indices_with_values[X_test[column]]
    counts = counts.fillna(0)
    counts = counts.as_matrix()
    data_processed_fold.loc[X_test.index, [column+'COUNTS']] = counts
    
    indices_with_values = X_train[y_train == 1][column].value_counts()
    clicks = indices_with_values[X_test[column]]
    clicks = clicks.fillna(0)
    clicks = clicks.as_matrix()
    data_processed_fold.loc[X_test.index, [column+'CLICKS']] = clicks
    
for column in X_train.columns:
    data_processed_fold[column+'RATIO'] = (
        data_processed_fold[column+'CLICKS'] + 1) / (data_processed_fold[column+'COUNTS'] + 2)
    data_processed_fold = data_processed_fold.fillna(0)
    
X_train_pro_fold, X_test_pro_fold, y_train_pro_fold, y_test_pro_fold = train_test_split(
    data_processed_fold.iloc[:, 1:], data_processed_fold.iloc[:, 0], test_size=0.3, random_state=241)

In [13]:
k_values = [1, 2, 4, 8, 16, 32, 64]
best_auc = 0
best_k = 0
for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train_pro_fold, y_train_pro_fold)
    predict = clf.predict_proba(X_test_pro_fold)
    auc = roc_auc_score(y_test_pro_fold.astype(float), predict[:, 1])
    if auc > best_auc:
        best_auc = auc
        best_k = k
print "Наилучшее качество:", best_auc, "Достигается при числе соседей k =", best_k

Наилучшее качество: 0.778147084258 Достигается при числе соседей k = 16


### Вывод: добавление пар -- неплохое средство поднять немного качество

## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [22]:
?GridSearchCV

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score
clf = DecisionTreeClassifier()
params = {'max_depth': [2, 4, 8, 16, 32], 'min_samples_leaf': [1, 2, 4, 8, 16, 32, 64]}
grid = GridSearchCV(clf, params, score_func=roc_auc_score)

In [30]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_processed.iloc[:, 1:], data_processed.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

In [31]:
grid.fit(X_train, y_train)
print 'Лучшая совокупность параметров:', grid.best_params_
print 'Лучший AUC-ROC:', grid.best_score_
print 'AUC-ROC на тесте с этими параметрами:', roc_auc_score(y_test, grid.predict_proba(X_test)[:, 1])

Лучшая совокупность параметров: {'max_depth': 8, 'min_samples_leaf': 4}
Лучший AUC-ROC: 0.999040892842
AUC-ROC на тесте с этими параметрами: 0.572223048894


#### Совсем не выглядит, как переобучение

#### А посмотрим на данные с фолдингом

In [32]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_processed_fold.iloc[:, 1:], data_processed_fold.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

In [33]:
grid.fit(X_train, y_train)
print 'Лучшая совокупность параметров:', grid.best_params_
print 'Лучший AUC-ROC:', grid.best_score_
print 'AUC-ROC на тесте с этими параметрами:', roc_auc_score(y_test, grid.predict_proba(X_test)[:, 1])

Лучшая совокупность параметров: {'max_depth': 16, 'min_samples_leaf': 32}
Лучший AUC-ROC: 0.950911151801
AUC-ROC на тесте с этими параметрами: 0.782742478907


#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score
clf = RandomForestClassifier()
params = {'n_estimators': [1, 2, 4, 8, 16, 32, 64]}
grid = GridSearchCV(clf, params, score_func=roc_auc_score)

In [35]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_processed.iloc[:, 1:], data_processed.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

In [38]:
grid.fit(X_train, y_train)
print 'Лучшая совокупность параметров:', grid.best_params_
print 'Лучший AUC-ROC:', grid.best_score_
print 'AUC-ROC на тесте с этими параметрами:', roc_auc_score(y_test, grid.predict_proba(X_test)[:, 1])

Время: 14.0 секунд
Лучшая совокупность параметров: {'n_estimators': 32}
Лучший AUC-ROC: 0.999215275961
AUC-ROC на тесте с этими параметрами: 0.636883761369


#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [39]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_processed_fold.iloc[:, 1:], data_processed_fold.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

In [40]:
grid.fit(X_train, y_train)
print 'Лучшая совокупность параметров:', grid.best_params_
print 'Лучший AUC-ROC:', grid.best_score_
print 'AUC-ROC на тесте с этими параметрами:', roc_auc_score(y_test, grid.predict_proba(X_test)[:, 1])

Лучшая совокупность параметров: {'n_estimators': 32}
Лучший AUC-ROC: 0.962769203941
AUC-ROC на тесте с этими параметрами: 0.826106248127


### Вывод: используя счетчики, необходимо использовать фолдинг, так как можно знатно переобучиться. Объяеняется это тем, что в счетчиках содержится информация об ответах.